### What is the range of the data (time-wise)?

In [4]:

import pandas as pd

# Load the Excel file
file_path = "Template Data.xlsx"
xls = pd.ExcelFile(file_path)

# Load the data
df = pd.read_excel(xls, sheet_name='Sheet2')

# Convert date columns to datetime format
df['Start Date'] = pd.to_datetime(df['Start Date'])
df['End Date'] = pd.to_datetime(df['End Date'])

# Get the time range of the data
time_range = df['Start Date'].min(), df['End Date'].max()
time_range


(Timestamp('2014-09-02 00:00:00'), Timestamp('2022-08-03 00:00:00'))

### Can we map a student to a course map?

In [6]:

# Check unique identifiers for students and courses
student_id_col = "Empl ID"
course_id_col = "Class Nbr"

# Check if a student can be mapped to multiple courses
student_course_mapping = df[[student_id_col, course_id_col]].drop_duplicates()

# Count the number of courses per student
courses_per_student = student_course_mapping[student_id_col].value_counts()

# Display basic stats about student-course mapping
courses_per_student.describe()


count    354.000000
mean      16.799435
std        7.385531
min        1.000000
25%       13.000000
50%       14.000000
75%       25.000000
max       41.000000
Name: count, dtype: float64

### Can it be brought into a simple data model?

In [8]:

# Check unique programs and academic plans
unique_programs = df["Prim Prog"].nunique()
unique_academic_plans = df["Acad Plan"].nunique()

# Count the number of courses per academic plan
courses_per_academic_plan = df.groupby("Acad Plan")["Class Nbr"].nunique()

# Display statistics
unique_programs, unique_academic_plans, courses_per_academic_plan.describe()


(12,
 17,
 count     17.000000
 mean      40.470588
 std       37.190586
 min        1.000000
 25%       15.000000
 50%       32.000000
 75%       52.000000
 max      153.000000
 Name: Class Nbr, dtype: float64)

### Are there 'odd' results where not expected (i.e. Marks)?

In [10]:

# Identify outlier marks (e.g., grades that are unusually high or low)
df['Official Grade'] = pd.to_numeric(df['Official Grade'], errors='coerce')

q1 = df["Official Grade"].quantile(0.25)
q3 = df["Official Grade"].quantile(0.75)
iqr = q3 - q1

# Define outliers as values outside 1.5 * IQR range
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

outlier_marks = df[(df["Official Grade"] < lower_bound) | (df["Official Grade"] > upper_bound)]

# Count and display outliers
num_outliers = outlier_marks.shape[0]

num_outliers, outlier_marks.head()


(282,
     Term  Class Nbr  Class Section Subject Catalog  \
 4   1221       4120            700    DBAS    3075   
 26  1221       4120            700    DBAS    3075   
 45  1218       4292            700    DBAS    2101   
 56  1218       4292            700    DBAS    2101   
 66  1218       2255            700    DBAS    2010   
 
                      Course Title Start Date   End Date Instructor Name  \
 4   Intro to Statistical Learning 2022-01-06 2022-02-11      Smith,John   
 26  Intro to Statistical Learning 2022-01-06 2022-02-11      Smith,John   
 45                 Data Reporting 2021-10-21 2021-12-16      Smith,John   
 56                 Data Reporting 2021-10-21 2021-12-16      Smith,John   
 66             Database Design II 2021-11-10 2021-12-16      Smith,John   
 
     Empl ID    Student Name  Official Grade Grading Basis Enrollment Dt  \
 4   1669518       Jose Mack            45.0           GRD    2021-12-01   
 26  1659712     John Kramer            42.0        

### Exception Reports: Failing Students, Incomplete Courses, etc.

In [12]:

# Identify failing students (grades < 60)
failing_students = df[df["Official Grade"] < 60]

# Count failing students
num_failing_students = failing_students["Empl ID"].nunique()

# Identify students with incomplete courses (null grade)
incomplete_courses = df[df["Official Grade"].isnull()]
students_with_incomplete = incomplete_courses["Empl ID"].nunique()

num_failing_students, students_with_incomplete


(92, 313)

### What are the average and standard deviation of marks for programs, courses, and students?

In [14]:

# Calculate average and standard deviation of marks for programs, courses, and students
avg_std_by_program = df.groupby("Prim Prog")["Official Grade"].agg(["mean", "std"])
avg_std_by_course = df.groupby("Class Nbr")["Official Grade"].agg(["mean", "std"])
avg_std_by_student = df.groupby("Empl ID")["Official Grade"].agg(["mean", "std"])

avg_std_by_program, avg_std_by_course, avg_std_by_student


(                mean        std
 Prim Prog                      
 BZIA       85.677215  13.942223
 CETN       76.909091  11.372101
 CYBR       83.541463  18.023903
 FSAD       93.571918   8.447837
 IFTG       58.071429  22.410383
 ITDA       88.164820  17.862916
 ITDB       86.726644  16.180782
 ITESP      93.546218   5.952848
 ITGE             NaN        NaN
 ITPR       81.536213  20.717616
 ITSM       87.629464  16.534692
 ITWP       85.857364  19.131243,
                 mean        std
 Class Nbr                      
 1663             NaN        NaN
 1685       32.000000        NaN
 1704             NaN        NaN
 1876       75.400000  25.553306
 1877       85.214286  17.603228
 ...              ...        ...
 5710        5.000000        NaN
 5711       73.000000        NaN
 5712       66.000000        NaN
 6354       87.000000        NaN
 6472       80.500000  27.073973
 
 [555 rows x 2 columns],
               mean        std
 Empl ID                      
 1222238  94.095238

### Top Teachers and Courses with Failures or Incomplete PNP

In [16]:

# Identify top teachers with failures or incomplete courses
top_teachers_failures = df[df["Official Grade"] < 60].groupby("Instructor Name").size().sort_values(ascending=False)

# Identify top courses with failures or incompletes
top_courses_failures = df[df["Official Grade"] < 60].groupby("Class Nbr").size().sort_values(ascending=False)

# Identify courses with the most failing or incomplete grades
top_courses_failing_or_incomplete = df[df["Official Grade"].isnull() | (df["Official Grade"] < 60)]     .groupby("Class Nbr").size().sort_values(ascending=False)

top_teachers_failures, top_courses_failures, top_courses_failing_or_incomplete


(Instructor Name
 James,Jessie         31
 Cooke,Adeanna        24
 Kelly,Grace          24
 Gandhi,Mahatma       20
 King,Billie Jean     18
 Tian,Xiao            17
 McCartney,Paul       15
 Gates,William        14
 Jackson.Michael      12
 Lennon,John          11
 Darwin,Charles       11
 Einstein,Albert       9
 Orwell,George         9
 Coyote, Wiley         9
 Hitchcock.Alfred      8
 Smith,John            8
 Fudd, Elmer           8
 Steward,Rod           7
 Cruise,Tom            7
 Rutkowska,Joanna      7
 Cohen,Sacha Baron     7
 Wilson,Donna          4
 Presley,Elvis         3
 Jobs,Steve On         3
 Armstrong,Neil        3
 Branson,Richard       3
 Huxley,Aldous         2
 Pitt,Brad             1
 Murdoch,Rupert        1
 Drover,Franklin       1
 Thatcher,Margaret     1
 Wales,Jimmy           1
 Walesa,Lech           1
 Andrews,Julie         1
 dtype: int64,
 Class Nbr
 4291    6
 3314    4
 4344    4
 4296    4
 3323    4
        ..
 3894    1
 3891    1
 3819    1
 3818   